## Keyword Extraction: 
~~Not to use, sent vs prop is enough~~
Reuse, sent vs prop make mistakes
Compare accuracy.

In [ ]:
# !pip install yake

In [3]:
import yake
kw_extractor = yake.KeywordExtractor()
# text = """spaCy is an open-source software library for advanced natural language processing, written in the programming languages Python and Cython. The library is published under the MIT license and its main developers are Matthew Honnibal and Ines Montani, the founders of the software company Explosion."""
# text = """Who is the director of ?"""
# text = """Who directed ?"""
# text = """What is the MPAA film rating of?"""
# text = """Let me know what  looks like."""
# text = """Who directed ?"""
# text = """Let me know what  looks like"""
text = """Show me a picture of ."""
language = "en"
max_ngram_size = 8
deduplication_threshold = 0.9
numOfKeywords = 1
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
     print(kw)

('Show me a picture', 0.04940384002065631)


## Similarity Comparison

### Load Property

In [4]:
import pickle
with open('P2label.pickle', 'rb') as handle:
    P2label = pickle.load(handle)
with open('label2P.pickle', 'rb') as handle:
    label2P = pickle.load(handle)
labels = list(P2label.values())

### TODO
### 

In [5]:
import numpy as np

In [7]:
labels[:5]

['assessment', 'country of origin', 'IMDb ID', 'instance of', 'award received']

In [5]:
label2P['follows']

'P155'

In [6]:
import pandas as pd

Name_Qwiki_Qid = pd.read_csv('name_Qwiki_Qid.tsv', sep='\t')
Name_Qwiki_Qid[-5:]

,Str,Wiki,Qid
150147,record,<http://www.wikidata.org/entity/Q107435521>,Q107435521
150148,Gujarati languages,<http://www.wikidata.org/entity/Q5617043>,Q5617043
150149,empire on which the sun never sets,<http://www.wikidata.org/entity/Q1211369>,Q1211369
150150,Finnish National Land Survey place type,<http://www.wikidata.org/entity/Q106589826>,Q106589826
150151,papal election,<http://www.wikidata.org/entity/Q29102902>,Q29102902


In [7]:
Name2Qid = dict(zip(Name_Qwiki_Qid['Str'], Name_Qwiki_Qid['Qid']))
Qid2Name = dict(zip(Name_Qwiki_Qid['Qid'], Name_Qwiki_Qid['Str']))
# Name2Qid = dict(zip(Name_Qwiki_Qid['Str'], Name_Qwiki_Qid['Qid']))
# Name2Qid = dict(zip(Name_Qwiki_Qid['Str'], Name_Qwiki_Qid['Qid']))

In [8]:
texts = [
    """What is the original language of ?""",
    """What is the MPAA film rating of ?""",
    """Who is the director of ?"""
]

### Try Spacy: Not to use

In [292]:
import spacy
nlp = spacy.load("en_core_web_md")  # make sure to use larger package!

def spacy_sent_sim(nlp, sent1, sent2):
    doc1 = nlp(sent1)
    doc2 = nlp(sent2)
    return doc1.similarity(doc2)


In [ ]:
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.4.1/en_core_web_lg-3.4.1-py3-none-any.whl
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.4.1/en_core_web_md-3.4.1-py3-none-any.whl
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.4.1/en_core_web_sm-3.4.1-py3-none-any.whl

In [300]:
sent1 = "Who directed ?"
sent2 = "director"
# print()
print(sent1, "<->", sent2, spacy_sent_sim(nlp, sent1, sent2))

Who directed ? <-> director 0.16556292927078123


### Try Sentence-Transformer Hugging_face

#### Load Model

In [ ]:
# !pip install -U sentence-transformers

In [ ]:
# from sentence_transformers import SentenceTransformer, util
# sentences = ["I'm happy", "I'm full of happiness"]

# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# #Compute embedding for both lists
# embedding_1= model.encode(sentences[0], convert_to_tensor=True)
# embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

# util.pytorch_cos_sim(embedding_1, embedding_2)




In [2]:
def Sent_Trans_sim(nlp, sent1, sent2):
  embedding_1= nlp.encode(sent1, convert_to_tensor=True)
  embedding_2 = nlp.encode(sent2, convert_to_tensor=True)
  return util.pytorch_cos_sim(embedding_1, embedding_2)

In [1]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/Users/nanfangwuyu/opt/anaconda3/envs/Classifier/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Handeling Labels

In [324]:
lbls = []
for label in labels:
    lbl = label.split(' ')
    lbls.append(lbl)
lbls.sort(key=lambda x: len(x), reverse=True)
lbls[:10]

[['located', 'in', 'or', 'next', 'to', 'body', 'of', 'water'],
 ['original', 'language', 'of', 'film', 'or', 'TV', 'show'],
 ['contributor', 'to', 'the', 'creative', 'work', 'or', 'subject'],
 ['located',
  'in',
  'the',
  'present-day',
  'administrative',
  'territorial',
  'entity'],
 ['located', 'in', 'the', 'administrative', 'territorial', 'entity'],
 ['said', 'to', 'be', 'the', 'same', 'as'],
 ['languages', 'spoken,', 'written', 'or', 'signed'],
 ['language', 'of', 'work', 'or', 'name'],
 ['has', 'works', 'in', 'the', 'collection'],
 ['takes', 'place', 'in', 'fictional', 'universe']]

In [8]:
labels = list(P2label.values())

special_conditions_reverse = {'description': '<http://schema.org/description>',
       'label': '<http://www.w3.org/2000/01/rdf-schema#label>',
       'tag': '<http://ddis.ch/atai/tag>', 'rating': '<http://ddis.ch/atai/rating>'}
extension_common = ['description', 'label', 'tag', 'rating']
extension_recommend = ['recommend']
extension_multimedia = ['look like'] # seem; appear; look like; look; resemble; bear resemblance to; be like.
labels.extend(extension_common)
labels.extend(extension_recommend)
labels.extend(extension_multimedia)

In [9]:
label_embeddings = model.encode(labels)

#### Encode Texts

In [402]:
# Redefine
texts = ["Who is the screenwriter of The Masked Gang: Cyprus?"]
texts = ["What is the MPAA film rating of"]
texts = ["What does Julia Roberts look like?"]
texts = ["Let me know what <> looks like"]
# texts = ["What is the box office of The Princess and the Frog?"]
# texts = ["Who directed ?"]

In [403]:
text_embeddings = model.encode(texts)

In [404]:
results = [util.pytorch_cos_sim(text_embeddings[0], label) for label in label_embeddings]

In [405]:
sort_res = results.copy()
sort_res.sort(reverse=True)
rank = 0
pred = labels[results.index(sort_res[0])]
for r in range(5):
    print(labels[results.index(sort_res[r])], sort_res[r])

## TODO
## If tensor < th? ask if you ask ...?

look like tensor([[0.4246]])
depicts tensor([[0.3722]])
interested in tensor([[0.3359]])
description tensor([[0.3075]])
image tensor([[0.3011]])


#### Function to Predict Label

In [10]:
def label_prediction(questions, use_keywords=True, use_expanding=False):
    keywords = questions
    if use_keywords:
        keywords = []
        for q in questions:
            keyword = custom_kw_extractor.extract_keywords(q)
            if not keyword:
                print("Use whole question")
                keywords.append(q)
            else:
                print("Keyword: ", keyword[0][0])
                keywords.append(keyword[0][0])

        print("Keywords: ", keywords)
    
    question_embeddings = model.encode(keywords)
    # print(question_embeddings)
    preds = []
    for embd in question_embeddings:
        results = [util.pytorch_cos_sim(embd, label) for label in label_embeddings]
        pred = labels[results.index(max(results))]
        preds.append(pred)
    return preds

In [ ]:
## TODO
## Calculate accuracy

## SPARQL Search

### API, Importing Library, and Data load

In [12]:
text = "Who directed The Bridge on the River Kwai?"
Q = "The Bridge on the River Kwai"
P = "director"

In [11]:
from rdflib.namespace import Namespace, RDF, RDFS, XSD
from rdflib.term import URIRef, Literal
import csv
import json
import networkx as nx
import rdflib
import pywikibot

In [13]:
graph = rdflib.Graph()
graph.parse('Datasets/14_graph.nt', format='turtle')

<Graph identifier=Nb1c037a5d39c4761a6d41a8b490d1644 (<class 'rdflib.graph.Graph'>)>

### Name to Qid, Label to Pid

In [100]:
def Qwiki2name_func(Qwiki: str):
    if Qwiki == None:
        return None
    if not Qwiki.startswith('http'):
        Qwiki = 'http://www.wikidata.org/entity/{}'.format(Qwiki)
    sql = \
        """
    prefix wdt: <http://www.wikidata.org/prop/direct/> 
    prefix wd: <http://www.wikidata.org/entity/> 
    SELECT ?lbl
    WHERE{
    <%s> rdfs:label ?lbl.
    FILTER(LANG(?lbl)='en')
    }

    """ % (Qwiki)

    tl = list(graph.query(sql))
    if len(tl)>0:
        for t in tl:
            for m in t:
                print(str(m))
                return str(m)
    else:
        return None

def name2Qid_func(name: str, func=1):
    if name == None:
        return None
    if func==1:
    ### TODO
    ### Change to SPARQL
    ### applymap(lambda x: x.replace('–', '-'))
    ### ?Qid rdf:label "name"@en --> result1
    ### if len > 1
    ###     ?Qid rdf:label "name"@en
    ###     ?Qid instance of  film --> result2
    ###     if len >= 1
    ###         random pick one from result2
    ###     else: random pick one from result1
    ### elif len = 1
    ### else None
        sql1 = \
        """
    prefix wdt: <http://www.wikidata.org/prop/direct/> 
    prefix wd: <http://www.wikidata.org/entity/> 
    SELECT ?Qid
    WHERE{
    ?Qid rdfs:label "%s"@en .
    }

    """ % (name)

        sql2 = \
        """
    prefix wdt: <http://www.wikidata.org/prop/direct/> 
    prefix wd: <http://www.wikidata.org/entity/> 
    SELECT ?Qid
    WHERE{
    ?Qid rdfs:label "%s"@en.
    ?Qid wdt:P31/wdt:P279* wd:Q11424
    
    }

    """ % (name)
    


    # """
    # prefix wdt: <http://www.wikidata.org/prop/direct/> 
    # prefix wd: <http://www.wikidata.org/entity/> 
    # SELECT ?Qid
    # WHERE{
    #     {?Qid wdt:P31 wd:Q11424 ."}
    #        UNION
    #         {?Qid rdfs:label "%s"@en .
    #         }
            
    #     }

    # """ % (name)
    

        Qids = []
        tl = list(graph.query(sql1))
        for t in tl:
            for m in t:
                Qids.append(str(m).removeprefix("http://www.wikidata.org/entity/"))
        # print(len(Qids))
        if len(Qids) == 0:
            print('Name2Qid failed, name: {}'.format(name))
            return None
        elif len(Qids) == 1:
            return Qids[0]
        
        elif len(Qids) > 1:
            Qids2 = []
            tl2 = list(graph.query(sql2))
            # print("TL:", tl2)
            for t in tl2:
                for m in t:
                    Qids2.append(str(m).removeprefix("http://www.wikidata.org/entity/"))
            if len(Qids2) >= 1:
                return np.random.choice(Qids2)
            else: 
                # print(Qids)
                return np.random.choice(Qids)
        
        
        
        
    else:
        if name in Name2Qid.keys():
            # "Star Wars: Episode VI - Return of the Jedi"
            return Name2Qid[name]
        else:
            # name = name.replace('-', '–')
            # if name in Name2Qid.keys():
            #     return Name2Qid[name]
            print('Name2Qid failed, name: {}'.format(name))
            return None

    # site = pywikibot.Site("en", "wikipedia")
    # page = pywikibot.Page(site, name)
    # item = pywikibot.ItemPage.fromPage(page)
    # return item.getID()

def label2Pid_func(label):
    if label in extension_multimedia:
        return label2P['image']

    if label in label2P.keys():
        return label2P[label]
    else:
        print('Label2Pid failed.')
        return None

In [15]:
# Qid = name2Qid_func(Q) # Q188718
Qid = name2Qid_func("Hamlet")
Qid

'Q5645443'

In [16]:
Pid = label2Pid_func(P) # P57

### Temp

In [58]:
# cast of Moon
temp_0 = """

    PREFIX ddis: <http://ddis.ch/atai/> 
    PREFIX wd: <http://www.wikidata.org/entity/> 
    PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
    PREFIX schema: <http://schema.org/> 
    
    SELECT ?lbl WHERE {
        ?movie rdfs:label "Moon"@en .
        ?movie wdt:P161 ?cast .
        ?cast rdfs:label ?lbl .
    }
    ''')]
"""

In [59]:
temp_1 = """
prefix wdt: <http://www.wikidata.org/prop/direct/> 
prefix wd: <http://www.wikidata.org/entity/> 
SELECT ?item ?label 
WHERE{
  ?item wdt:P31 wd:Q55488 .
  ?item wdt:P131 wd:Q72 .
  ?item rdfs:label ?label .
  FILTER(LANG(?label) = "en").
}
LIMIT 1
"""

# search results for: 
#     '?' 'BELONGS TO' 'RAILWAY'
# AND '?' 'PLACE IN' 'ZURICH'


In [ ]:
### Formal Temp ###

temp_2 = \
"""
prefix wdt: <http://www.wikidata.org/prop/direct/> 
prefix wd: <http://www.wikidata.org/entity/> 
SELECT ?label
WHERE{
  wd:%s wdt:%s ?item .
  ?item rdfs:label ?label .
  FILTER(LANG(?label) = "en").
}
""" % (Q_wiki, P_wiki)

In [61]:
temp_3 = \
"""
prefix wdt: <http://www.wikidata.org/prop/direct/> 
prefix wd: <http://www.wikidata.org/entity/> 
SELECT *
WHERE{
  ?Q1 ?P ?Q2 .
  ?Q1 rdfs:label ?Q1l .
  FILTER(LANG(?Q1l) = "en").
  ?P rdfs:label ?Pl .
  FILTER(LANG(?Pl) = "en").
  ?Q2l rdfs:label ?Q2l .
  FILTER(LANG(?Q2l) = "en").

}
LIMIT 2
"""

In [ ]:
tl = list(graph.query(temp_3))
for e in tl:
    print(str(tl))

In [63]:
temp_4 = \
"""
prefix wdt: <http://www.wikidata.org/prop/direct/> 
prefix wd: <http://www.wikidata.org/entity/> 
SELECT *
WHERE{
  ?Q1 wdt:P57 ?Q2 .

}
LIMIT 2
"""

In [64]:
for s in graph.query(temp_4):
    print(s)

(rdflib.term.URIRef('http://www.wikidata.org/entity/Q360509'), rdflib.term.URIRef('http://www.wikidata.org/entity/Q1000825'))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q360509'), rdflib.term.URIRef('http://www.wikidata.org/entity/Q3122116'))


In [65]:
# Another way to query

wdt = Namespace('http://www.wikidata.org/prop/direct/')
wd = Namespace('http://www.wikidata.org/entity/')
lbl = Namespace('http://www.w3.org/2000/01/rdf-schema#')
# tr = (wd[Q_wiki], wdt[P_wiki], None)
tr = (None, lbl['label'], '')
for t in graph.triples(tr):
    print(t)

In [66]:
temp_5 = \
"""
prefix wdt: <http://www.wikidata.org/prop/direct/> 
prefix wd: <http://www.wikidata.org/entity/> 
SELECT *
WHERE{
  ?Q1 ?P ?Q2 .

}
LIMIT 2
"""

In [ ]:
for s in graph.query(temp_5):
    print(s)

### SQL Generation

#### Distinguish question type

In [214]:
recommend_labels = ['recommend']
multimedia_labels = ['image', 'look like']
special_conditions_reverse = {'description': '<http://schema.org/description>',
       'label': '<http://www.w3.org/2000/01/rdf-schema#label>',
       'tag': '<http://ddis.ch/atai/tag>', 'rating': '<http://ddis.ch/atai/rating>'}


question_type = 'common'
pred = P
def distinguish_question_type(pred):
    question_type = 'common'
    if pred in special_conditions_reverse.keys():
        question_type = 'special'
        
    elif pred in recommend_labels:
        question_type = 'recommend'

    elif pred in multimedia_labels:
        question_type = 'multimedia'
    return question_type
question_type = distinguish_question_type(pred)

#### Special Questions

In [85]:
def answer_special_questions(Q, P):
    Qid = name2Qid_func(Q)
    Puri = special_conditions_reverse[P]
    common_sql = \
"""
prefix wdt: <http://www.wikidata.org/prop/direct/> 
prefix wd: <http://www.wikidata.org/entity/> 
SELECT ?item
WHERE{
  wd:%s %s ?item .
}
""" % (Qid, Puri)
    results = [str(s) for s, in graph.query(common_sql)]
    if len(results):
      # print(common_sql)
      return results[0]
    else:
      print('Answer special questions failed, sql: {}'.format(common_sql))
      return None

In [90]:
answer_special_questions(Q, 'description')

'Good Neighbors'

#### Common Questions

In [43]:
def answer_common_questions(Q, P, func='cro'):
    # fuc: fac, emb, cro
    answer = answer_crowdsourcing_questions(Q, P)
    if answer:
      return {'type': 'cro', 'answer': answer}
    else:
      answer_fac = answer_factual_questions(Q, P)
      answer_emb = answer_embedding_questions(Q, P)
      if answer_fac == None and answer_emb == None:
        return None
      elif answer_fac != None and answer_emb == None:
        return {'type': 'fac', 'answer': answer_fac}
      elif answer_fac == None and answer_emb != None:
        return {'type': 'emb', 'answer': answer_emb}
      else:
        return {'type': 'fac and emb', 'answer': (answer_emb, answer_fac)}
      

In [21]:
### Test
answer_common_questions(Q, P)

'David Lean'

##### Common-Fac

In [196]:
def answer_factual_questions(Q, P, func='fac'):

    Qid = name2Qid_func(Q)
    Pid = label2Pid_func(P)
    common_sql = \
"""
prefix wdt: <http://www.wikidata.org/prop/direct/> 
prefix wd: <http://www.wikidata.org/entity/> 
SELECT ?label
WHERE{
  wd:%s wdt:%s ?item .
  ?item rdfs:label ?label .
  FILTER(LANG(?label) = "en").
}
""" % (Qid, Pid)
    results = [str(s) for s, in graph.query(common_sql)]
    if len(results):
      # print(common_sql)
      return results[:3]
    else:
      print('Answer factual questions failed.'.format(common_sql))
      return None

In [126]:
Q, P, answer_factual_questions(Q, P)
### TODO multiple answers return 3

('Good Neighbors', 'genre', ['art film', 'comedy-drama'])

In [72]:
answer_factual_questions(Qwiki2name_func('Q1168152'), P2label['P161'])

The Blue Butterfly


'Marc Donato'

##### Common-Emb

In [22]:
from sklearn.metrics import pairwise_distances

In [25]:
WD = rdflib.Namespace('http://www.wikidata.org/entity/')
WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
DDIS = rdflib.Namespace('http://ddis.ch/atai/')
RDFS = rdflib.namespace.RDFS
SCHEMA = rdflib.Namespace('http://schema.org/')

In [26]:
entity_emb = np.load('Datasets/EmbeddingData/entity_embeds.npy')
relation_emb = np.load('Datasets/EmbeddingData/relation_embeds.npy')

In [27]:
with open('Datasets/EmbeddingData/entity_ids.del', 'r') as ifile:
    ent2id = {rdflib.term.URIRef(ent): int(idx) for idx, ent in csv.reader(ifile, delimiter='\t')}
    id2ent = {v: k for k, v in ent2id.items()}
with open('Datasets/EmbeddingData/relation_ids.del', 'r') as ifile:
    rel2id = {rdflib.term.URIRef(rel): int(idx) for idx, rel in csv.reader(ifile, delimiter='\t')}
    id2rel = {v: k for k, v in rel2id.items()}

In [28]:
ent2lbl = {ent: str(lbl) for ent, lbl in graph.subject_objects(RDFS.label)}
lbl2ent = {lbl: ent for ent, lbl in ent2lbl.items()}

In [31]:
def answer_embedding_questions(Q, P, func='emb'):

    Qid = name2Qid_func(Q)
    Pid = label2Pid_func(P)
    head = entity_emb[ent2id[WD[Qid]]]
    pred = relation_emb[rel2id[WDT[Pid]]]
    # add vectors according to TransE scoring function.
    lhs = head + pred
    # compute distance to *any* entity
    dist = pairwise_distances(lhs.reshape(1, -1), entity_emb).reshape(-1)
    # find most plausible entities
    most_likely = dist.argsort()

    print([(id2ent[idx][len(WD):], ent2lbl[id2ent[idx]], dist[idx], i) for i, idx in enumerate(most_likely[:3])])
    emb_result = ent2lbl[id2ent[most_likely[0]]]

    return emb_result

In [33]:
Q, P, answer_embedding_questions(Q, P)

[('Q26118', 'Jack Hawkins', 2785.5608, 0), ('Q55260', 'David Lean', 2934.9756, 1), ('Q3127608', 'Harold Goodwin', 2940.063, 2)]


('The Bridge on the River Kwai', 'director', 'Jack Hawkins')

##### Common-Cro

In [35]:
cro_df = pd.read_csv('Datasets/CrowdData/Clean_Cro.tsv', sep='\t')

In [158]:
results = cro_df.loc[(cro_df['Input1ID']=='wd:Q217010') & (cro_df['Input2ID']=='wdt:P2142')]

In [163]:
list(results['GroupAnswers'])

[False, False, False]

In [83]:
results[results['FixPosition'].str.contains('Object') & (~results['FixValue'].str.contains('Empty'))]

,HITId,HITTypeId,Title,Reward,AssignmentId,WorkerId,AssignmentStatus,WorkTimeInSeconds,LifetimeApprovalRate,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue,GroupAnswers,TrueCount,FalseCount,Agreement
22,8,7QT,Is this triple correct or incorrect?,$0.50,37,928UJANWZ12,Submitted,240,98%,wd:Q217010,wdt:P2142,698491347,2.0,INCORRECT,Object,698491348,False,0,3,0.24


In [179]:
# need_to_fix = cro_df.loc[(cro_df['Input1ID']=='wd:Q11621') & (cro_df['Input2ID']=='wdt:P2142')]
for i in cro_df.index:
    Qid = cro_df['Input1ID'].loc[i].removeprefix('wd:')
    Pid = cro_df['Input2ID'].loc[i].removeprefix('wdt:')
    print(answer_crowdsourcing_questions(Qid, Pid, use_id=True))

Q11621 P2142
792910554
{'response_type': 'correct', 'result': '792910554', 'TrueCount': 2, 'FalseCount': 1, 'Agreement': 0.24}
Q11621 P2142
792910554
{'response_type': 'correct', 'result': '792910554', 'TrueCount': 2, 'FalseCount': 1, 'Agreement': 0.24}
Q11621 P2142
792910554
{'response_type': 'correct', 'result': '792910554', 'TrueCount': 2, 'FalseCount': 1, 'Agreement': 0.24}
Q603545 P2142
4300000
{'response_type': 'correct', 'result': '4300000', 'TrueCount': 3, 'FalseCount': 0, 'Agreement': 0.24}
Q603545 P2142
4300000
{'response_type': 'correct', 'result': '4300000', 'TrueCount': 3, 'FalseCount': 0, 'Agreement': 0.24}
Q603545 P2142
4300000
{'response_type': 'correct', 'result': '4300000', 'TrueCount': 3, 'FalseCount': 0, 'Agreement': 0.24}
Q16911843 P577
2014-01-18
{'response_type': 'correct', 'result': '2014-01-18', 'TrueCount': 2, 'FalseCount': 1, 'Agreement': 0.24}
Q16911843 P577
2014-01-18
{'response_type': 'correct', 'result': '2014-01-18', 'TrueCount': 2, 'FalseCount': 1, 'Agr

In [173]:
def answer_crowdsourcing_questions(Q, P, func='cro', use_id=False):

    
    if use_id:
        Qid = Q
        Pid = P
    else:
        Qid = name2Qid_func(Q)
        Pid = label2Pid_func(P)

    # print(Qid, Pid)
    
    micro_task = cro_df.loc[(cro_df['Input1ID']=='wd:{}'.format(Qid)) & (cro_df['Input2ID']=='wdt:{}'.format(Pid))]
    if len(micro_task) > 0:
        #   GroupAnswers | TrueCount | FalseCount | Agreement
        #   FixPosition | FixValue 
        #   Input3ID
        GroupAnswers, TrueCount, FalseCount, Agreement, FixPositions, FixValues, Input3ID = (
        list(micro_task['GroupAnswers'])[0], 
        list(micro_task['TrueCount'])[0],
        list(micro_task['FalseCount'])[0],
        list(micro_task['Agreement'])[0],
        list(micro_task['FixPosition']),
        list(micro_task['FixValue']),
        str(list(micro_task['Input3ID'])[0]),
        )
        print(Input3ID)
        response_type = 'correct'
        if GroupAnswers == True:
            # use the provided answer

            if not Input3ID.startswith('wd:'):
                # return string
                cro_result = Input3ID
            else:
                # transfer id to label
                Qid = Input3ID[3:]
                cro_result = Qwiki2name_func(Qid)
        else:
           

            # fix or mention
            fixable = micro_task[micro_task['FixPosition'].str.contains('Object') & (~micro_task['FixValue'].str.contains('Empty'))]
            # if FixPositions != 'Empty' and FixValues != 'Empty':
            # both_not_empty = micro_task[(~micro_task['FixPosition'].str.contains('Empty')) & (~micro_task['FixValue'].str.contains('Empty'))]

            if len(fixable) > 0:
                fix_values = fixable['FixValue'].unique()
                value = fix_values[0]
                # fix
                response_type = 'fix'
                if value.startswith('wd:'):
                    value = value[3:]
                if value.startswith('Q'):
                    cro_result = Qwiki2name_func(value)
                else:
                    cro_result = value
                
                
            else:
                if Input3ID[:3] != 'wd:':
                    # return string
                    cro_result = Input3ID
                else:
                    # transfer id to label
                    Qid = Input3ID[3:]
                    cro_result = Qwiki2name_func(Qid)
                # just mention
                # The answer in knowledge graph is wrong -- according to the crowd, ~~~
                response_type = 'mention'

                

        return {
        'response_type': response_type,
        'result': cro_result,
        'TrueCount': TrueCount, 
        'FalseCount': FalseCount,
        'Agreement': Agreement,
        }

    else:
        print('what')
        return None


#### Multimedia question

In [216]:
with open('Datasets/ImageData/id2Code.pickle', 'rb') as handle:
    id2Code = pickle.load(handle)

In [217]:
def answer_multimedia_questions(Q, P):
    Qid = name2Qid_func(Q)
    print(Qid)

    sql = \
        """
    prefix wdt: <http://www.wikidata.org/prop/direct/> 
    prefix wd: <http://www.wikidata.org/entity/> 
    SELECT ?id
    WHERE{
    wd:%s wdt:P345 ?id .
    }

    """ % (Qid)

    id = ''
    tl = list(graph.query(sql))
    for t in tl:
        for m in t:
            id = str(m)
            break
    print(id)
    if id in id2Code:
        return "image:{}".format(id2Code[id])
    else:
        print("Id is not in id to code dictionary .")
        return None

#### Recommend question

In [98]:
def answer_recommend_questions(Qs: list):
  
    Qids = [name2Qid_func(Q) for Q in Qs]
    print("Qids:", Qids)
    Ps = ['genre', 'publication date', 'director', 'set in period']
    Pids_withpre = ['wdt:{}'.format(label2Pid_func(P)) for P in Ps]
    tag_rdf = special_conditions_reverse['tag']
    Pids_withpre.append(tag_rdf)


    features_all = [[] for i in range(len(Pids_withpre))]
    for i, Qid in enumerate(Qids):
      features = fetch_features_from_sql(Qid, Pids_withpre)
      for j, feature in enumerate(features):
        features_all[j].append(feature)

    analysis = analysis_recommendation(features_all)

    recommend_map = {}
    recommend_map.update({"director": analysis[2]})
    recommend_map.update({"set_in_period": analysis[3]})
    recommend_map.update({"genre": analysis[0]})
    recommend_map.update({"publication_date": analysis[1]})
    
    # print(analysis_results)
    recommendations = search_recommendations(Qs, recommend_map, Pids_withpre)

    map = recommend_map
    map['director'] = Qwiki2name_func(map['director'])
    map['set_in_period'] = Qwiki2name_func(map['set_in_period'])
    map['genre'] = Qwiki2name_func(map['genre'])

      
    return recommend_map, recommendations

In [95]:
def search_recommendations(Qs, map, Pids_withpre):
    print(map)
    # genres, public_dates, directors, set_in_period, tags
    
    # lines_genre = "\n".join(["?movie %s wd:%s ." % (Pids_withpre[0], name2Qid_func(list(map['genre'])[i])) for i in range(len(list(map['genre'])))]) if map['genre'] else ''
    lines_genre = "?movie %s <%s> ." % (Pids_withpre[0], map['genre']) if map['genre'] else ''


    lines_publication_date = """
    ?movie %s ?pubdate .
    FILTER (?pubdate >= "%s-01-01"^^xsd:date)
    FILTER (?pubdate < "%s-12-31"^^xsd:date)
    """ % (Pids_withpre[1], list(map['publication_date'])[0], list(map['publication_date'])[1]) if map['publication_date'] else ''
    
    # lines_director = "\n".join(["?movie %s wd:%s ." % (Pids_withpre[2], name2Qid_func(list(map['director'])[i])) for i in range(len(list(map['director'])))]) if map['director'] else ''
    lines_director = "?movie %s <%s> ." % (Pids_withpre[2], map['director']) if map['director'] else ''


    # lines_period = """
    # ?movie wdt:P2408 ?period .
    # ?period rdfs:label "%s"@en.

    # """ % (list(map['set_in_period'])[0]) if map['set_in_period'] else ''

    lines_period = """
    ?movie wdt:P2408 <%s> .

    """ % (map['set_in_period']) if map['set_in_period'] else ''
    
    # lines_tag = "\n".join(["?movie %s wd:%s ." % (Pids_withpre[4], name2Qid_func(list(map['tag'])[i])) for i in range(len(list(map['tag'])))]) if map['tag'] else ''

    def genreate_sql(lines_genre, lines_publication_date, lines_director, lines_period):
        sql = \
            """
        prefix wdt: <http://www.wikidata.org/prop/direct/> 
        prefix wd: <http://www.wikidata.org/entity/> 
        prefix ps: <http://www.wikidata.org/prop/statement/>
        prefix p: <http://www.wikidata.org/prop/>
        SELECT ?lb1
        WHERE{
        
        %s\n%s\n%s\n%s

        ?movie wdt:P31/wdt:P279* wd:Q11424 .
        ?movie rdfs:label ?lb1 .
        FILTER(LANG(?lb1) = "en").
        }
        LIMIT 6
        """ % (lines_genre, lines_publication_date, lines_director, lines_period)
        # print(sql)
        return(sql)
    sql = genreate_sql(lines_genre, lines_publication_date, lines_director, lines_period)
    # print(sql)
    movies = set([str(s) for s, in graph.query(sql)])
    movies = movies.difference(set(Qs))

    if len(movies) == 0:
        sql = genreate_sql(lines_genre, lines_publication_date, lines_director, '')
        movies = set([str(s) for s, in graph.query(sql)])
        movies = movies.difference(set(Qs))
        if len(movies) == 0:
            sql = genreate_sql(lines_genre, lines_publication_date, '', '')
            movies = set([str(s) for s, in graph.query(sql)])
            movies = movies.difference(set(Qs))
            if len(movies) == 0:
                sql = genreate_sql(lines_genre, '', '', '')
                movies = set([str(s) for s, in graph.query(sql)])
                movies = movies.difference(set(Qs))

    
    # print(movies)
    if len(movies) > 0:
        if len(movies) > 3:
            return list(movies)[:3]
        else:
            return list(movies)
    else:
        return None
    # return list(movies)[:3] if len(movies) > 0 else None
    

In [96]:
def fetch_features_from_sql(Qid, Pids_withpre):
    sqls = []
    for i, Pid in enumerate(Pids_withpre):
      sql = \
          """
    prefix wdt: <http://www.wikidata.org/prop/direct/> 
    prefix wd: <http://www.wikidata.org/entity/> 
    SELECT ?item1
    WHERE{
    wd:%s %s ?item1 .
    ?item1 rdfs:label ?lb1 .
    FILTER(LANG(?lb1) = "en").
    }
    """ % (Qid, Pid)
    # lb1
      sqls.append(sql)

    sqls[1] = \
          """
    prefix wdt: <http://www.wikidata.org/prop/direct/> 
    prefix wd: <http://www.wikidata.org/entity/> 
    SELECT ?lb1
    WHERE{
    wd:%s %s ?item1 .
    ?item1 rdfs:label ?lb1 .
    FILTER(LANG(?lb1) = "en").
    }
    """ % (Qid, Pids_withpre[1])
    genres = [str(s) for s, in graph.query(sqls[0])]
    public_dates = [str(s) for s, in graph.query(sqls[1])]
    directors = [str(s) for s, in graph.query(sqls[2])]
    set_in_period = [str(s) for s, in graph.query(sqls[3])]
    tags = [str(s) for s, in graph.query(sqls[4])]


    # print("{}\n{}\n{}\n{}\n{}\n".format(genres, public_dates, directors, set_in_period, tags))
    return genres, public_dates, directors, set_in_period, tags

In [97]:
from collections import Counter

def analysis_recommendation(features_all):
    # Ps = ['genre', 'publication date', 'director', 'set in period'] + tag

    genres, public_dates, directors, set_in_period, tags = features_all
    def analysis_intersection(features):
        if len(features) == 0:
            return None
        intersection = set(features[0])
        for l in features[1:]:
            intersection = intersection & set(l)
        return list(intersection)[0] if len(intersection) > 0 else None

    def analysis_min_max(features):
        if len(features) == 0:
            return None
        union = set(features[0])
        for l in features[1:]:
            union = union | set(l)
        if len(union) == 0:
            return None
        dates = [int(date[:4]) for date in union]
        return min(dates), max(dates)

    def analysis_most_common(features):
        if len(features) == 0:
            return None
        li = features[0]
        for l in features[1:]:
            li.extend(l)
        if len(li) == 0:
            return None
        
        # for i, period in enumerate(li):
        #     if period[-1:] != 's':
        #         li[i] = str((int(period) // 10) * 10)+'s'
        # print(li)
        counter = Counter(li)
        # print(counter.most_common(1)[0][0])
        return counter.most_common(1)[0][0]

    analysis = analysis_most_common(genres), \
            analysis_min_max(public_dates), \
                analysis_intersection(directors), \
                    analysis_most_common(set_in_period), \
                        analysis_intersection(tags)

    
    return analysis


    # print(analysis_public_dates(['2018-08','2020-02', '2010-02', '2019']))
    
    return None

In [114]:
X = [
        "Recommend movies like Nightmare on Elm Street, Friday the 13th, and Halloween.",
        "Given that I like The Lion King, Pocahontas, and The Beauty and the Beast, can you recommend some movies?",
    ]

Q_ents = [
    "A Nightmare on Elm Street",
    "Friday the 13th",
    "Halloween",
]


Q_ents = [
    "The Lion King",
    "Pocahontas",
    "Beauty and the Beast",
]

# ['Q36479', 'Q218894', 'Q19946102']
# ['Q36479', 'Q27688562', 'Q495442']

# Q_ents = [
#     "Hamlet",
#     "Othello",
# ]

# Q_ents = [
#     "Hornet's Nest",
# ] # publication date


map, rec = answer_recommend_questions(Q_ents)
map, rec
# TODO handle {'node description', 'node label', 'cast member', 'Rotten Tomatoes ID', 'YouTube video ID'} condition

Qids: ['Q36479', 'Q218894', 'Q19946102']
{'director': None, 'set_in_period': 'http://www.wikidata.org/entity/Q6756', 'genre': 'http://www.wikidata.org/entity/Q842256', 'publication_date': None}
1607
musical film


({'director': None,
  'set_in_period': '1607',
  'genre': 'musical film',
  'publication_date': None},
 ['Hum Tum', 'Jonas Brothers: The 3D Concert Experience', 'Sleeping Beauty'])

In [225]:
# sql2 = \
#         """
#     prefix wdt: <http://www.wikidata.org/prop/direct/> 
#     prefix wd: <http://www.wikidata.org/entity/> 
#     SELECT ?Qid
#     WHERE{
#     ?Qid rdfs:label "%s"@en.
#     ?Qid wdt:P31/wdt:P279* wd:Q11424
    
#     }

#     """ % ("Pocahontas")

# Qids2 = []
# tl2 = list(graph.query(sql2))
# print("TL:", tl2)

TL: [(rdflib.term.URIRef('http://www.wikidata.org/entity/Q218894'),)]


#### Decide which function

In [189]:
def answer_question(question_type, **kwargs):
    if question_type == 'common':
        return answer_common_questions(kwargs['Q'], kwargs['P'])
    elif question_type == 'recommend':
        return answer_recommend_questions(kwargs['Qs']) #list
    elif question_type == 'multimedia':
        return answer_multimedia_questions(kwargs['Q'], kwargs['P'])
    elif question_type == 'special':
        return answer_special_questions(kwargs['Q'], kwargs['P'])

In [336]:
### Test
answer_question(question_type, Q=Q, P=P)

1
Answer common questions failed, sql: 
prefix wdt: <http://www.wikidata.org/prop/direct/> 
prefix wd: <http://www.wikidata.org/entity/> 
SELECT ?label
WHERE{
  wd:Q40791 wdt:P18 ?item .
  ?item rdfs:label ?label .
  FILTER(LANG(?label) = "en").
}



### Simple Question Answer Test

#### Factual Questions

In [201]:
fqs_origin = [
    "Who is the director of Good Will Hunting?",
    "Who directed The Bridge on the River Kwai?",
    "Who is the director of Star Wars: Episode VI - Return of the Jedi?",
]

Q_ents = [
    "Good Will Hunting",
    "The Bridge on the River Kwai",
    "Star Wars: Episode VI - Return of the Jedi",
] # should be extract by model

fqs_mask = [
    "Who is the director of <>?",
    "Who directed <>?",
    "Who is the director of <>?",
]

In [202]:
preds = label_prediction(fqs_mask)
preds

Keyword:  director
Keyword:  directed
Keyword:  director
Keywords:  ['director', 'directed', 'director']


['director', 'director', 'director']

In [203]:
answers = []

for Q, P, question in (zip(Q_ents, preds, fqs_origin)):
    # Qid, Pid = name2Qid_func(Q), label2Pid_func(P)
    question_type = distinguish_question_type(P)
    answer = answer_question(question_type=question_type, Q=Q, P=P)
    full_answer = full_answer_generation(question_type, answer)
    print("Question:", question)
    print("Answer:", full_answer)

Q193835 P57
what
[('Q528949', 'Harmony Korine', 2721.4583, 0), ('Q483118', 'Ben Affleck', 2832.0962, 1), ('Q25186', 'Gus Van Sant', 2863.7493, 2)]
Question: Who is the director of Good Will Hunting?
Answer: Take the embeddings as reference, I support Harmony Korine is the answer you want. However, the answer is different if I search it in knowledge graph: Gus Van Sant.
Q188718 P57
what
[('Q26118', 'Jack Hawkins', 2785.5608, 0), ('Q55260', 'David Lean', 2934.9756, 1), ('Q3127608', 'Harold Goodwin', 2940.063, 2)]
Question: Who directed The Bridge on the River Kwai?
Answer: Hmm, a hard question! But I know the answer is likely to be Jack Hawkins, provided by embeddings. Different answer would be fetched from knowledge graph: David Lean.
Q181803 P57
what
[('Q311319', 'Frank Oz', 2904.5396, 0), ('Q38222', 'George Lucas', 2919.8804, 1), ('Q240872', 'Lawrence Kasdan', 2987.016, 2)]
Question: Who is the director of Star Wars: Episode VI - Return of the Jedi?
Answer: According to embeddings, th

#### Embedding Questions

These questions don't have a exact answer. Use embeddings to find most similar ones.

In [206]:
eqs_origin = [
    "Who is the screenwriter of The Masked Gang: Cyprus?",
    "What is the MPAA film rating of Weathering with You?",
    "What is the genre of Good Neighbors?",
]

Q_ents = [
    "The Masked Gang: Cyprus",
    "Weathering with You",
    "Good Neighbors",
]

eqs_mask = [
    "Who is the screenwriter of <>?",
    "What is the MPAA film rating of <>?",
    "What is the genre of <>?"
]

In [207]:
preds = label_prediction(eqs_mask)
preds

Keyword:  screenwriter
Keyword:  MPAA film rating
Keyword:  genre
Keywords:  ['screenwriter', 'MPAA film rating', 'genre']


['screenwriter', 'MPA film rating', 'genre']

In [209]:
answers = []

for Q, P, question in (zip(Q_ents, preds, eqs_origin)):
    # Qid, Pid = name2Qid_func(Q), label2Pid_func(P)
    question_type = distinguish_question_type(P)
    answer = answer_question(question_type=question_type, Q=Q, P=P)
    full_answer = full_answer_generation(question_type, answer)
    print("Question:", question)
    print("Answer:", full_answer)

Q7750525 P58
what
Answer factual questions failed.
[('Q5058838', 'Cengiz Küçükayvaz', 2568.8137, 0), ('Q3328043', 'Murat Aslan', 2613.7378, 1), ('Q6812382', 'Melih Ekener', 2728.1128, 2)]
Question: Who is the screenwriter of The Masked Gang: Cyprus?
Answer: I find the answer to this question from embeddings: Cengiz Küçükayvaz.
Q59692464 P1657
what
[('Q18665339', 'PG-13', 5618.7075, 0), ('Q18665334', 'PG', 6026.137, 1), ('Q18665344', 'R', 6102.3394, 2)]
Question: What is the MPAA film rating of Weathering with You?
Answer: According to embeddings, the answer to this question is PG-13. However, the answer is different if I search it in knowledge graph: NC-17.
Q3110682 P136
what
[('Q130232', 'drama', 3599.8713, 0), ('Q859369', 'comedy-drama', 3604.7146, 1), ('Q157443', 'comedy film', 3616.0613, 2)]
Question: What is the genre of Good Neighbors?
Answer: Take the embeddings as reference, I support drama is the answer you want. However, there would be multiple answers in knowledge graph: art

#### Multimedia Questions

In [210]:
mqs_origin = [
    "Show me a picture of Halle Berry.",
    "What does Julia Roberts look like?",
    "Let me know what Sandra Bullock looks like.",
    "Show me a picture of Bruce Willis.",
]

Q_ents = [
    "Halle Berry",
    "Julia Roberts",
    "Sandra Bullock",
    "Bruce Willis",

]# TODO if many ents a question

mqs_mask = [
    "Show me a picture of <>.",
    "What does <> look like?",
    "Let me know what <> looks like.",
    "Show me a picture of <>.",
]


In [211]:
preds = label_prediction(mqs_mask)

Keyword:  Show me a picture
Use whole question
Use whole question
Keyword:  Show me a picture
Keywords:  ['Show me a picture', 'What does <> look like?', 'Let me know what <> looks like.', 'Show me a picture']


In [212]:
preds

['image', 'look like', 'look like', 'image']

In [218]:
answers = []

for Q, P, question in (zip(Q_ents, preds, mqs_origin)):
    # Qid, Pid = name2Qid_func(Q), label2Pid_func(P)
    question_type = distinguish_question_type(P)
    answer = answer_question(question_type=question_type, Q=Q, P=P)
    full_answer = full_answer_generation(question_type, answer)
    print("Question:", question)
    print("Answer:", full_answer)

Q1033016
nm0000932
Question: Show me a picture of Halle Berry.
Answer: image:0353/rm3257480192.jpg
Q40523
nm0000210
Question: What does Julia Roberts look like?
Answer: image:3739/rm3651656960.jpg
Q40791
nm0000113
Question: Let me know what Sandra Bullock looks like.
Answer: image:1050/rm3423772416.jpg
Q2680
nm0000246
Question: Show me a picture of Bruce Willis.
Answer: image:1926/rm1037117952.jpg


#### Recommendation Questions

In [228]:
X = [
        "Recommend movies similar to Hamlet and Othello.",
        "Recommend movies like Nightmare on Elm Street, Friday the 13th, and Halloween.",
        "Given that I like The Lion King, Pocahontas, and The Beauty and the Beast, can you recommend some movies?",
    ]

Q_ents = [
    "A Nightmare on Elm Street",
    "Friday the 13th",
    "Halloween",
]


Q_ents = [
    "The Lion King",
    "Pocahontas",
    "Beauty and the Beast",
]

# ['Q36479', 'Q218894', 'Q19946102']
# ['Q36479', 'Q27688562', 'Q495442']

# Q_ents = [
#     "Hamlet",
#     "Othello",
# ]

# Q_ents = [
#     "Hornet's Nest",
# ] # publication date


# map, rec = answer_recommend_questions(Q_ents)
# map, rec

# handle {'node description', 'node label', 'cast member', 'Rotten Tomatoes ID', 'YouTube video ID'} condition

answer = answer_question('recommend', Qs = Q_ents)
print(full_answer_generation('recommend', answer))

Qids: ['Q27044293', 'Q218894', 'Q19946102']
{'director': None, 'set_in_period': 'http://www.wikidata.org/entity/Q6756', 'genre': 'http://www.wikidata.org/entity/Q130232', 'publication_date': None}
1607
drama
I would recommend you these movies: Linger, What We Wanted, Kill the Irishman, and you may have interest in movies in the drama genre, with a background period in 1607.


#### Crowdsourcing Questions

These questions do have an answer, but in crowding source dataset.

In [222]:
cqs_origin = [
    "What is the box office of The Princess and the Frog?",
    "Can you tell me the publication date of Tom Meets Zizou?",
    "Who is the executive producer of X-Men: First Class?",
]

Q_ents = [
    "The Princess and the Frog",
    "Tom Meets Zizou",
    "X-Men: First Class",
]

cqs_mask = [
    "What is the box office of <>?",
    "Can you tell me the publication date of <>?",
    "Who is the executive producer of <>?"
]



In [223]:
preds = label_prediction(cqs_mask)
preds

Keyword:  box office
Keyword:  publication date
Keyword:  executive producer
Keywords:  ['box office', 'publication date', 'executive producer']


['box office', 'publication date', 'executive producer']

In [224]:
answers = []

for Q, P, question in (zip(Q_ents, preds, cqs_origin)):
    # Qid, Pid = name2Qid_func(Q), label2Pid_func(P)
    question_type = distinguish_question_type(P)
    answer = answer_question(question_type=question_type, Q=Q, P=P)
    full_answer = full_answer_generation(question_type, answer)
    print("Question:", question)
    print("Answer:", full_answer)

Q171300 P2142
267000000
Question: What is the box office of The Princess and the Frog?
Answer: Good question. I suppose the answer is 267000000, suggested by crowd. The crowd had an inter-rater agreement of 0.24, in the related batch. The answer distribution of this micro task was 2 support and 1 reject.
Q1410031 P577
2010-10-01
Question: Can you tell me the publication date of Tom Meets Zizou?
Answer: According to crowd, the fixed answer to this question is 2011-01-01. The crowd had an inter-rater agreement of 0.04, in the related batch. The answer distribution of this micro task was 0 support and 3 reject.
Q223596 P1431
wd:Q457180
Sheryl Lee Ralph
Question: Who is the executive producer of X-Men: First Class?
Answer: Hmm, a hard question! But I know the answer is probably to be Sheryl Lee Ralph, provided by crowd. The crowd had an inter-rater agreement of 0.26, in the related batch. The answer distribution of this micro task was 2 support and 1 reject.


### Answer Generation

In [92]:
def full_answer_generation_fai(answer):
    # Failed to answer question
    choices = \
    [
        "I'm sorry that I can't answer this question to you now, can you change another one?",
        "This question is a bit of hard for me, ask me another one please!",
        "Oops! It's difficult for me to show you answer to this question, ask me a new one!",
    ]

    return np.random.choice(choices)

full_answer_generation_fai(None)

In [94]:
def full_answer_generation_mul(answer):
    # Image code
    return answer

# full_answer_generation_mul(None)

In [229]:
def full_answer_generation_rec(answer):
    # Failed to answer question
    map, rec = answer
    choices_1 = \
    [
        "I would recommend you these movies: {}".format(', '.join(rec)),
        "I would like to recommend these movies to you: {}".format(', '.join(rec)),
        "I suppose you will also like: {}".format(', '.join(rec)),
    ]

    
    
    choice_genre = "in the {} genre".format(map['genre']) if map['genre'] else None
    choice_director = "directed by {}".format(map['director']) if map['director'] else None
    if map["publication_date"]:
        if map["publication_date"][0] == map["publication_date"][1]:
            choice_publicated = "pulicated around {}".format(map["publication_date"][0])
        else:
            choice_publicated = "pulicated from {} to {}".format(map["publication_date"][0], map["publication_date"][1])
    else:
        choice_publicated = None
    
    choice_period = "with a background period around {}".format(map["set_in_period"]) if map["set_in_period"] else None

    merge = []
    if choice_genre:
        merge.append(choice_genre)
    if choice_director:
        merge.append(choice_director)
    if choice_publicated:
        merge.append(choice_publicated)
    if choice_period:
        merge.append(choice_period)
    other = ', '.join(merge)

    choices_2 = \
        [
            ". By aspects, movies {} may speak to you.".format(other),
            ". Besides, movies {} may to your appetite.".format(other),
            ", and you may have interest in movies {}.".format(other),
        ]

    c1 = np.random.choice(choices_1)
    c2 = np.random.choice(choices_2)

    return "{}{}".format(c1, c2)

answer_temp = ({'director': 'Wtier',
  'set_in_period': '1607',
  'genre': 'musical film',
  'publication_date': (1900, 1900)},
 ['Hum Tum', 'Jonas Brothers: The 3D Concert Experience', 'Sleeping Beauty'])
full_answer_generation_rec(answer_temp)

'I would like to recommend these movies to you: Hum Tum, Jonas Brothers: The 3D Concert Experience, Sleeping Beauty, and you may have interest in movies in the musical film genre, directed by Wtier, pulicated around 1900, with a background period around 1607.'

In [122]:
def full_answer_generation_spe(answer):
    # special

    # choices = \
    #     [
    #         "The answer to this question is {}.".format(answer),
    #         "I know this, {}.".format(answer),
    #         "Good question, I suppose the answer is {}.".format(answer),
    #         "Hmm, a hard question, but I know the answer is {}".format(answer),
    #         "I support {} is the answer you want.".format(answer),
    #     ]

    choices = \
        [
            "The answer to this question is: {}.".format(answer),
            "I know this, {}.".format(answer),
            "Good question, I suppose the answer is: {}.".format(answer),
            "Hmm, a hard question, but I know the answer: {}".format(answer),
            "I support {} is the answer you want.".format(answer),
        ]

    return np.random.choice(choices)

In [133]:
def full_answer_generation_fac(answer):
    # factual

    if len(answer) == 1:
        answer = answer[0]


        choices = \
            [
                "The answer to this question is {}.".format(answer),
                "I know this, {}.".format(answer),
                "Good question, I suppose the answer is {}.".format(answer),
                "Hmm, a hard question, but I know the answer is {}".format(answer),
                "I support {} is the answer you want.".format(answer),
            ]
    elif len(answer) == 2:

        choices = \
            [
                "The answers to this question are: {}.".format(' and '.join(answer)),
                "I know this, {}.".format(' and '.join(answer)),
                "Good question, I suppose the answers will be: {}.".format(' and '.join(answer)),
                "Hmm, a hard question, but I know the answers: {}".format(' and '.join(answer)),
                "I support {} are the answers you want.".format(' and '.join(answer)),
            ]
    
    elif len(answer) > 2:

        choices = \
            [
                "The answers to this question are: {}.".format(', '.join(answer)),
                "I know this, {}.".format(', '.join(answer)),
                "Good question, I suppose the answers will be: {}.".format(', '.join(answer)),
                "Hmm, a hard question, but I know the answers: {}".format(', '.join(answer)),
                "I support {} are the answers you want.".format(', '.join(answer)),
            ]

    return np.random.choice(choices)

full_answer_generation_fac(['drama']),\
full_answer_generation_fac(['drama', 'musical']),\
full_answer_generation_fac(['drama', 'musical', 'song'])

('The answer to this question is drama.',
 'I support drama and musical are the answers you want.',
 'I support drama, musical, song are the answer you want.')

In [141]:
def full_answer_generation_emb(answer):
    # embedding

    choices = \
        [
            "According to embeddings, the answer to this question is {}.".format(answer),
            "I find the answer to this question from embeddings: {}.".format(answer),
            "Good question, I suppose the answer is {}, suggested by embeddings.".format(answer),
            "Hmm, a hard question! But I know the answer is likely to be {}, provided by embeddings.".format(answer),
            "Take the embeddings as reference, I support {} is the answer you want.".format(answer),
        ]
    

    return np.random.choice(choices)

full_answer_generation_emb('drama')

'According to embeddings, the answer to this question is drama.'

In [146]:
def full_answer_generation_bot(answer):
    # factual and embedding
    # {'type': 'fac and emb', 'answer': [answer_emb, answer_fac]}

    answer_emb, answer_fac = answer
    full_emb = full_answer_generation_emb(answer_emb)

    if len(answer_fac) == 1:
        answer_fac = answer_fac[0]
        
        if answer_emb == answer_fac:
            full_fac = np.random.choice([
                "The answer is the same if I find the answer in knowledge graph.",
                "I also get the same answer from knowledge graph, what a coincidence!",
            ])
            

        else:
            full_fac = np.random.choice([
            "However, I find another possible answer in knowledge graph: {}".format(answer_fac),
            "However, the answer is different if I search it in knowledge graph: {}.".format(answer_fac),
            "Different answer would be fetched from knowledge graph: {}.".format(answer_fac)
        ])

        
    else:
        full_fac = np.random.choice([
                "However, there would be multiple answers in knowledge graph: {}.".format(', '.join(answer_fac)),
                "While multiple answers would be fetched from knowledge graph: {}.".format(', '.join(answer_fac)),
            ])


       

    return "{} {}".format(full_emb, full_fac)

full_answer_generation_bot(('drama', ['drama'])),\
full_answer_generation_bot(('drama', ['musical'])),\
full_answer_generation_bot(('drama', ['drama', 'musical']))

('According to embeddings, the answer to this question is drama. I also get the same answer from knowledge graph, what a coincidence!',
 'According to embeddings, the answer to this question is drama. Different answer would be fetched from knowledge graph: musical.',
 'Good question, I suppose the answer is drama, suggested by embeddings. While multiple answers would be fetched from knowledge graph: drama, musical.')

In [181]:
def full_answer_generation_cro(answer: dict):
    # crowdsourcing
    # {
        # 'response_type': response_type, # correct fix mention
        # 'result': cro_result, # label
        # 'TrueCount': TrueCount, # int
        # 'FalseCount': FalseCount, # int
        # 'Agreement': Agreement, # float
        # }

    # London - according to the crowd, who had an inter-rater agreement of 0.72 in this batch. 
    # The answer distribution for this specific task was 2 support votes and 1 reject vote. 

    for key in answer.keys():
        if answer[key] == None:
            return full_answer_generation_fai(None)

    choices_2 = [
            "The crowd had an inter-rater agreement of {}, in the related batch. ".format(answer['Agreement']) + 
            "The answer distribution of this micro task was {} support and {} reject.".format(answer['TrueCount'], answer['FalseCount']),
            "The inter-rater agreement of relevant batch is {}. ".format(answer['Agreement']) + 
            "The concrete distribution of relevant micro task was {} support and {} reject.".format(answer['TrueCount'], answer['FalseCount']),
        ]
    info_answer = np.random.choice(choices_2)

    if answer['response_type'] == 'correct':
        part_answer = np.random.choice([
            "Hmm, a hard question! But I know the answer is probably to be {}, provided by crowd.".format(answer['result']),
            "Good question. I suppose the answer is {}, suggested by crowd.".format(answer['result']),
            "According to crowd, the answer to this question is {}.".format(answer['result']),
            "I find the answer to this question from crowd: {}.".format(answer['result']),
            "The crowd suggested that the answer is: {}".format(answer['result'])
        ])

        
    elif answer['response_type'] == 'fix':
        part_answer = np.random.choice([
            "Hmm, a hard question! But I know the answer is probably to be {}, which is fixed by crowd.".format(answer['result']),
            "Good question. I suppose the answer is {}, fixed by crowd.".format(answer['result']),
            "According to crowd, the fixed answer to this question is {}.".format(answer['result']),
            "I find a fixed answer to this question from crowd: {}.".format(answer['result']),
            "The crowd suggested that the fixed answer is: {}".format(answer['result'])
        ])

    else:
        # mention
        part_answer = np.random.choice([
            "Hmm, a hard question! The answer is probably to be {} from knowledge graph. However, the crowd suggested that this relationship is wrong. Sorry that I don't have complete information. But I can show you some other information.".format(answer['result']),
            "Good question. The answer in knowledge graph is {}, but the crowd think it's wrong. I don't have enough information to fix it. Here is some other analysis:".format(answer['result']),
            "According to crowd, the answer to this question is wrong, although we can get it from knowledge graph: {}. Sorry that I don't have complete information. Here is some other information I want to show you.".format(answer['result']),
        ])
    
    

    return "{} {}".format(part_answer, info_answer)

full_answer_generation_cro({'response_type': 'correct', 'result': '267000000', 'TrueCount': 2, 'FalseCount': 1, 'Agreement': 0.24}),\
full_answer_generation_cro({'response_type': 'fix', 'result': '2011-01-01', 'TrueCount': 0, 'FalseCount': 3, 'Agreement': 0.04}),\
full_answer_generation_cro({'response_type': 'mention', 'result': '969023261', 'TrueCount': 0, 'FalseCount': 3, 'Agreement': 0.24}
)

('According to crowd, the answer to this question is 267000000. The crowd had an inter-rater agreement of 0.24, in the related batch. The answer distribution of this micro task was 2 support and 1 reject.',
 'The crowd suggested that the fixed answer is: 2011-01-01 The crowd had an inter-rater agreement of 0.04, in the related batch. The answer distribution of this micro task was 0 support and 3 reject.',
 "Good question. The answer in knowledge graph is 969023261, but the crowd think it's wrong. I don't have enough information to fix it. Here is some other analysis: The inter-rater agreement of relevant batch is 0.24. The concrete distribution of relevant micro task was 0 support and 3 reject.")

In [191]:
# question_type = 'common'
# pred = P
# if pred in special_conditions_reverse.keys():
#     question_type = 'special'
    
# elif pred in extension_recommend:
#     question_type = 'recommend'

# elif pred in extension_multimedia:
#     question_type = 'multimedia'
def full_answer_generation(question_type, answer):
    if not answer:
        return full_answer_generation_fai(answer)
    if question_type == 'multimedia':
        return full_answer_generation_mul(answer)
    elif question_type == 'recommend':
        return full_answer_generation_rec(answer)
    elif question_type == 'special':
        return full_answer_generation_spe(answer)
    elif question_type == 'common':
        if answer['type'] == 'cro':
            return full_answer_generation_cro(answer['answer'])
        elif answer['type'] == 'emb':
            return full_answer_generation_emb(answer['answer'])
        elif answer['type'] == 'fac':
            return full_answer_generation_fac(answer['answer'])
        elif answer['type'] == 'fac and emb':
            return full_answer_generation_bot(answer['answer'])
    return full_answer_generation_fai(answer)

## Agent

In [231]:
### Human Username: yijie.tong Password: rI5OtxGVTzWzwA
### AI Username: yijie.tong_bot Password: 3AiyYO7SLQ77MA
import time
import atexit
import getpass
import requests  # install the package via "pip install requests"
from collections import defaultdict
from io import BytesIO
from PIL import Image

# url of the speakeasy server
url = 'https://speakeasy.ifi.uzh.ch'
listen_freq = 3


class DemoBot:
    def __init__(self, username, password):
        self.agent_details = self.login(username, password)
        self.session_token = self.agent_details['sessionToken']
        self.chat_state = defaultdict(lambda: {'messages': defaultdict(dict), 'initiated': False, 'my_alias': None})

        atexit.register(self.logout)

    def listen(self):
        while True:
            # check for all chatrooms
            current_rooms = self.check_rooms(session_token=self.session_token)['rooms']
            print("Current rooms number: ", len(current_rooms))
            for room in current_rooms:
                # ignore finished conversations
                if room['remainingTime'] > 0:
                    room_id = room['uid']
                    if not self.chat_state[room_id]['initiated']:
                        # send a welcome message and get the alias of the agent in the chatroom
                        # self.post_message(room_id=room_id, session_token=self.session_token, message='Hi, you can send me any message and check if it is echoed in {} seconds.'.format(listen_freq))
                        self.post_message(room_id=room_id, session_token=self.session_token, message="Welcome! I'm chatbot MTon! You can ask me questions. Glad to communicate with you!")
                        self.chat_state[room_id]['initiated'] = True
                        self.chat_state[room_id]['my_alias'] = room['alias']

                    # check for all messages
                    all_messages = self.check_room_state(room_id=room_id, since=0, session_token=self.session_token)['messages']

                    # you can also use ["reactions"] to get the reactions of the messages: STAR, THUMBS_UP, THUMBS_DOWN

                    for message in all_messages:
                        if message['authorAlias'] != self.chat_state[room_id]['my_alias']:

                            # check if the message is new
                            if message['ordinal'] not in self.chat_state[room_id]['messages']:
                                self.chat_state[room_id]['messages'][message['ordinal']] = message
                                print('\t- Chatroom {} - new message #{}: \'{}\' - {}'.format(room_id, message['ordinal'], message['message'], self.get_time()))

                                ##### You should call your agent here and get the response message #####

                                answer = 'Got your message: \'{}\' at {}.'.format(message['message'], self.get_time())
                                answer = 'image:1050/rm3423772416.jpg'
                                self.post_message(room_id=room_id, session_token=self.session_token, message=answer, )
            time.sleep(listen_freq)

    def login(self, username: str, password: str):
        agent_details = requests.post(url=url + "/api/login", json={"username": username, "password": password}).json()
        print(agent_details.keys())
        print('- User {} successfully logged in with session \'{}\'!'.format(agent_details['userDetails']['username'], agent_details['sessionToken']))
        return agent_details

    def check_rooms(self, session_token: str):
        return requests.get(url=url + "/api/rooms", params={"session": session_token}).json()

    def check_room_state(self, room_id: str, since: int, session_token: str):
        return requests.get(url=url + "/api/room/{}/{}".format(room_id, since), params={"roomId": room_id, "since": since, "session": session_token}).json()

    def post_message(self, room_id: str, session_token: str, message: str):

        ##### Try multimedia #####


        tmp_des = requests.post(url=url + "/api/room/{}".format(room_id),
                                params={"roomId": room_id, "session": session_token}, data=message).json()
        if tmp_des['description'] != 'Message received':
            print('\t\t Error: failed to post message: {}'.format(message))

    def get_time(self):
        return time.strftime("%H:%M:%S, %d-%m-%Y", time.localtime())

    def logout(self):
        if requests.get(url=url + "/api/logout", params={"session": self.session_token}).json()['description'] == 'Logged out':
            print('- Session \'{}\' successfully logged out!'.format(self.session_token))


if __name__ == '__main__':
    username = 'yijie.tong_bot'
    # password = getpass.getpass('Password of the demo bot:')
    password = '3AiyYO7SLQ77MA'
    demobot = DemoBot(username, password)
    demobot.listen()






dict_keys(['userDetails', 'sessionToken', 'sessionId', 'startTime'])
- User yijie.tong_bot successfully logged in with session 'node01567rr220jgmc14xx30nmyvdmw437046'!
Current rooms number:  1
	- Chatroom f769fe7e-0d59-43ad-a4ed-f2836a3bbb87 - new message #0: 'Who is the director of Good Will Hunting?' - 04:35:59, 14-12-2022
Current rooms number:  1
Current rooms number:  1
Current rooms number:  1
Current rooms number:  1
Current rooms number:  1
Current rooms number:  1
Current rooms number:  1
Current rooms number:  1
Current rooms number:  1
Current rooms number:  1


KeyboardInterrupt: 